# Recommender model based on sklearn
https://medium.com/@sumanadhikari/building-a-movie-recommendation-engine-using-scikit-learn-8dbb11c5aa4b

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import linear_model

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pigidser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

/home/pigidser/anaconda3/envs/recsys/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/pigidser/anaconda3/envs/recsys/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head(3)

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,userid,itemid,rating
0,5.0,True,"10 4, 2016",B01CPNIEQG,Heather,These are my FAVORITE spices in my collection....,Must Add to your Spice kitchen!,1475539200,NaN,NaN,NaN,102179,37138,1.0
1,5.0,True,"03 1, 2016",B006F63M8U,Linda Odom,Add A package to my Coffee and it makes a good...,Milk Chocolate Swiis MIss Hot Cocoa Mix,1456790400,NaN,{'Size:': ' 60-0.73 oz Envelopes'},NaN,3625,17322,1.0
2,5.0,True,"06 26, 2016",B00112O8NG,DesertBlossom,"I love the Torani syrups, but not the prices o...","Love these sugar free syrups, but didn't love ...",1466899200,28,NaN,NaN,39495,5600,1.0


## Products

In [4]:
%%time
# Read products and create dataframe
f = open('meta_Grocery_and_Gourmet_Food.json', 'r') 
count = 0
data = []
while True: 
    count += 1
    line = f.readline()
    if not line: break
    data.append(json.loads(line))
f.close() 

prod = pd.DataFrame(data)

CPU times: user 21.5 s, sys: 2.13 s, total: 23.6 s
Wall time: 23.9 s


In [5]:
prod.drop(['date','feature','details','similar_item','tech1','fit'],axis=1, inplace=True)

In [6]:
# Leave products which are used in train and test
asin = np.concatenate([train.asin.values, test.asin.values])
prod = prod[(prod.asin.isin(asin))]
prod.reset_index(inplace=True)

In [7]:
%%time
train['reviewText'].fillna('',inplace=True)
train['summary'].fillna('',inplace=True)
item_reviews = dict()
for row in train[['asin','reviewText','summary']].iterrows():
    if row[1]['asin'] in item_reviews.keys():
        review = str(item_reviews[row[1]['asin']])
        review = review + ' ' + row[1]['reviewText'] + ' ' + row[1]['summary']
        item_reviews[row[1]['asin']] = review
    else:
        item_reviews[row[1]['asin']] = row[1]['reviewText'] + ' ' + row[1]['summary']


CPU times: user 1min 43s, sys: 79.2 ms, total: 1min 43s
Wall time: 1min 43s


In [8]:
df = pd.DataFrame.from_dict(item_reviews, orient='index')
df.reset_index(inplace=True)
df.columns = ['asin', 'review']
prod = prod.merge(df, on='asin', how='left')
prod.head(3)

,index,category,description,title,brand,rank,also_view,main_cat,price,asin,also_buy,image,review
0,10,"[Grocery & Gourmet Food, Beverages, Coffee, Te...",[Lipton Yellow Label Tea use only the finest t...,Lipton Yellow Label Tea (loose tea) - 450g,Lipton,"30,937inGroceryGourmetFood(","[B00CREXSHY, B001QTRGAQ, B000JSQK70, B002EYZM4...",Grocery,$12.46,4639725043,"[B00886E4K0, B00CREXSHY, B001QTRGAQ, B002EYZM4...",[https://images-na.ssl-images-amazon.com/image...,OK tea. TEA Best tea for my single cup coffee ...
1,11,"[Grocery & Gourmet Food, Beverages, Coffee, Te...",[Lipton Yellow Label Teabags uses a new way to...,Lipton Yellow Label Finest Blend Tea Bags 100 ...,Lipton,"15,487inGroceryGourmetFood(","[B000JSQDK4, B007ZI1SKG, B00286KM8E, B01LY2304...",Grocery,$12.98,4639725183,"[B000JSQDK4, B007ZI1SKG, 4639725043, B00KPFHOP...",[https://images-na.ssl-images-amazon.com/image...,Hard to find in the U.S.A. Exactly as describ...
2,15,"[Grocery & Gourmet Food, Beverages, Coffee, Te...","[20 Sachets, Empty contents into cup\nPour 8oz...",Organo Gold Cafe Supreme 100% Certified Ganode...,Organo Gold,"89,943inGroceryGourmetFood(","[B003CGJAIM, B00F3XJX6G, B01LVXI1GP, B00E1C3FJ...",Grocery,$29.90,5463213682,"[B00F3XJX6G, B003CY45VG, B003CGJAIM, B00E1C3FJ...",[https://images-na.ssl-images-amazon.com/image...,enjoy this coffee and have not been able to bu...


In [9]:
%%time
features = ['category','description','title','brand','also_view','main_cat','also_buy','review']

for feature in features:
    prod[feature] = prod[feature].fillna('')
    prod[feature] = prod[feature].astype('str')

def combine_features(row):
    return row['category'] + ' ' + row['description'] + ' ' + row['title'] + ' ' + row['brand'] \
        + ' ' + row['also_view'] + ' ' + row['main_cat'] + ' ' + row['also_buy'] + ' ' + row['review']

prod['combined_features'] = prod[features].apply(combine_features, axis=1)
prod.head(3)

CPU times: user 3.74 s, sys: 68.1 ms, total: 3.81 s
Wall time: 3.81 s


,index,category,description,title,brand,rank,also_view,main_cat,price,asin,also_buy,image,review,combined_features
0,10,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",['Lipton Yellow Label Tea use only the finest ...,Lipton Yellow Label Tea (loose tea) - 450g,Lipton,"30,937inGroceryGourmetFood(","['B00CREXSHY', 'B001QTRGAQ', 'B000JSQK70', 'B0...",Grocery,$12.46,4639725043,"['B00886E4K0', 'B00CREXSHY', 'B001QTRGAQ', 'B0...",[https://images-na.ssl-images-amazon.com/image...,OK tea. TEA Best tea for my single cup coffee ...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."
1,11,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",['Lipton Yellow Label Teabags uses a new way t...,Lipton Yellow Label Finest Blend Tea Bags 100 ...,Lipton,"15,487inGroceryGourmetFood(","['B000JSQDK4', 'B007ZI1SKG', 'B00286KM8E', 'B0...",Grocery,$12.98,4639725183,"['B000JSQDK4', 'B007ZI1SKG', '4639725043', 'B0...",[https://images-na.ssl-images-amazon.com/image...,Hard to find in the U.S.A. Exactly as describ...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."
2,15,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...","['20 Sachets', 'Empty contents into cup\nPour ...",Organo Gold Cafe Supreme 100% Certified Ganode...,Organo Gold,"89,943inGroceryGourmetFood(","['B003CGJAIM', 'B00F3XJX6G', 'B01LVXI1GP', 'B0...",Grocery,$29.90,5463213682,"['B00F3XJX6G', 'B003CY45VG', 'B003CGJAIM', 'B0...",[https://images-na.ssl-images-amazon.com/image...,enjoy this coffee and have not been able to bu...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe..."


In [10]:
import re

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [ ]:
%%time
prod['combined_features'] = prod['combined_features'].apply(clean_text)

In [95]:
prod.head(20)

CPU times: user 7.52 s, sys: 3.85 ms, total: 7.53 s
Wall time: 7.52 s


,index,category,description,title,brand,rank,also_view,main_cat,price,asin,also_buy,image,review,combined_features
0,10,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",['Lipton Yellow Label Tea use only the finest ...,Lipton Yellow Label Tea (loose tea) - 450g,Lipton,"30,937inGroceryGourmetFood(","['B00CREXSHY', 'B001QTRGAQ', 'B000JSQK70', 'B0...",Grocery,$12.46,4639725043,"['B00886E4K0', 'B00CREXSHY', 'B001QTRGAQ', 'B0...",[https://images-na.ssl-images-amazon.com/image...,OK tea. TEA Best tea for my single cup coffee ...,grocery gourmet food beverages coffee tea coco...
1,11,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",['Lipton Yellow Label Teabags uses a new way t...,Lipton Yellow Label Finest Blend Tea Bags 100 ...,Lipton,"15,487inGroceryGourmetFood(","['B000JSQDK4', 'B007ZI1SKG', 'B00286KM8E', 'B0...",Grocery,$12.98,4639725183,"['B000JSQDK4', 'B007ZI1SKG', '4639725043', 'B0...",[https://images-na.ssl-images-amazon.com/image...,Hard to find in the U.S.A. Exactly as describ...,grocery gourmet food beverages coffee tea coco...
2,15,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...","['20 Sachets', 'Empty contents into cup\nPour ...",Organo Gold Cafe Supreme 100% Certified Ganode...,Organo Gold,"89,943inGroceryGourmetFood(","['B003CGJAIM', 'B00F3XJX6G', 'B01LVXI1GP', 'B0...",Grocery,$29.90,5463213682,"['B00F3XJX6G', 'B003CY45VG', 'B003CGJAIM', 'B0...",[https://images-na.ssl-images-amazon.com/image...,enjoy this coffee and have not been able to bu...,grocery gourmet food beverages coffee tea coco...
3,42,"['Grocery & Gourmet Food', 'Sauces, Gravies & ...","['Mae Ploy Thai green curry paste.', 'Ingredie...","Mae Ploy Green Curry Paste, 14 oz",Mae Ploy,"39,261inGroceryGourmetFood(","['B002P8AQJ0', 'B000EICISA', 'B0091UW4QS', 'B0...",Grocery,NaN,9742356831,"['B000EICJWA', 'B000EICISA', 'B074ZMGQNT', 'B0...",[https://images-na.ssl-images-amazon.com/image...,"I had high hopes, but unfortunately, too salty...",grocery gourmet food sauces gravies marinades ...
4,44,"['Grocery & Gourmet Food', 'Cooking & Baking',...","[""This set is a great value from one of the gr...","Ateco Food Coloring Kit, 6 colors",Harold Import Company,">#257,139 in Kitchen & Dining (See Top 100 in ...","['B00KCUXRLC', 'B0026N2Y3U', 'B01BZ5U3Y6', 'B0...",Amazon Home,NaN,B00004S1C5,,NaN,"Do the job Five Stars Too waxy, not what i was...",grocery gourmet food cooking baking food color...
5,45,"['Grocery & Gourmet Food', 'Snack Foods', 'Jer...",['Love jerky but not the price? Make it cheap ...,Metalware Open Country Beef Jerky Works Kit,Nesco,">#341,157 in Kitchen & Dining (See Top 100 in ...","['B001795P3K', 'B01FL1FE1W', 'B000FFX642', 'B0...",Amazon Home,NaN,B00004W4VD,,[https://images-na.ssl-images-amazon.com/image...,Works perfect. Made strips and pepperoni stic...,grocery gourmet food snack foods jerky dried m...
6,48,"['Grocery & Gourmet Food', 'Candy & Chocolate'...","[""Effective relief. Fights coughs naturally. S...","Ricola Cough Suppressant Throat Drops, Swiss H...",Ricola,"30,623inGroceryGourmetFood(","['B002BX6TUY', 'B01IAI9D8I', 'B00AY6RO9U', 'B0...",Grocery,$3.13,B000052X2S,"['B002BX6TUY', 'B01B6CBRHK', 'B01IAI9D8I', 'B0...",[https://images-na.ssl-images-amazon.com/image...,Tasty! Five Stars The Swiss Herb flavor leaves...,grocery gourmet food candy chocolate hard cand...
7,49,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",['Biotene Dry Mouth Gum Natural Enzyme Protect...,Biotene Dry Mouth Gum Natural Enzyme Protectio...,Biotne,"484,760inGroceryGourmetFood(",,Grocery,NaN,B000052Y74,,NaN,As a dentist I see many patients that have dry...,grocery gourmet food candy chocolate chewing b...
8,50,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...","['Personality: Wise and lively, a breath of fr...",Traditional Medicinals Organic Breathe Easy Se...,Traditional Medicinals,"13,305inGroceryGourmetFood(","['B0016B63S0', 'B00LHGEL6K', 'B00KPTQGEA', 'B0...",Grocery,$28.68,B00005344V,"['B0009F3POE', 'B007IW25FQ', 'B0016B63S0', 'B0...",[https://imag

In [13]:
prod.to_csv('prod_combined_features.csv',index=False)

In [12]:
%%time
cv = CountVectorizer()
count_matrix = cv.fit_transform(prod["combined_features"])

CPU times: user 19.7 s, sys: 116 ms, total: 19.8 s
Wall time: 19.8 s


In [14]:
type(count_matrix)

scipy.sparse.csr.csr_matrix

In [15]:
%%time
cosine_sim = cosine_similarity(count_matrix, dense_output=False)

CPU times: user 4min 26s, sys: 2.37 s, total: 4min 29s
Wall time: 4min 29s


In [19]:
# print(cv.get_feature_names())гш
# print(count_matrix.toarray())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
prod[prod['asin']=='5463213682'].index[0]

2

In [102]:
def get_title_from_index(index):
    return prod[prod.index == index]['title'].values[0] + '(' + prod[prod.index == index]['asin'].values[0] + ')' 

def get_index_from_asin(asin):
    return prod[prod.asin == asin].index[0]

prod_user_likes = 'B00006FMLY'
prod_index = get_index_from_asin(prod_user_likes)
similar_prods =  list(enumerate(cosine_sim[prod_index].toarray()[0]))

sorted_similar_prods = sorted(similar_prods,key=lambda x:x[1],reverse=True)[1:]

i=0
print("Top 5 similar prods to "+prod_user_likes+" are:\n")
for element in sorted_similar_prods[1:]:
    print(get_title_from_index(element[0]),element[1])
    i=i+1
    if i>=5:
        break

Top 5 similar prods to B00006FMLY are:

Wilton 710-2082 Primary Sprinkle Set, 3.83-Ounce(B0055DCZCI) 0.5541631672020676
Wilton Edible Glitter Silver Stars,  0.4 oz.(B003KK42V0) 0.5396283858919453
Wilton Holiday Sprinkles Assortment, 6.8 oz.(B000JJO058) 0.5283893929423611
Wilton Black Sugar Pearls(B002QJ5SM8) 0.5179992178204765
Cake Mate Sprinkles Chocolate Flavored 26 Oz(B007ASS95I) 0.5110101681822214
